# Land Cover (C-factor) and Practices (P-factor)

This notebook provides an interactive tool to estimate the **C-factor (Cover Management)** and **P-factor (Support Practices)** used in the Universal Soil Loss Equation (USLE) and its revised versions (RUSLE, RUSLE2).  

**Disclaimer:**  
The C and P factor estimations used in this tool are based on **simplified assumptions and representative values** for illustrative purposes. While these values are not site-specific, they serve as a reference to ensure that **calibrated C and P values** (which users are encouraged to adjust based on local conditions) align with ranges reported in official RUSLE manuals.  

This tool aims to:  
1. Help users **better understand how management practices (C)** and **erosion control measures (P)** influence soil loss calculations.  
2. Provide a starting point for calibration, bridging theoretical guidelines with practical application.  

**Note:** Actual determination of these factors for specific sites requires detailed local data (e.g., crop rotations, tillage practices, terrace design) and adherence to established methodologies (e.g., USDA-NRCS handbooks). For precision, consider transitioning to advanced tools like **RUSLE2**.  

## Import (or load) the necessary libraries and iMPACT-tools

##### Import tools

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from ipywidgets import interactive, VBox, HBox, HTML # For layout
# Import the necessary iMPACTools (you can find these tools in the Python files stored in the */iMPACtools* folder)
os.chdir('..') # change the current working directory to the parent directory
from iMPACTools.interactive_plots import plot_interactive_CP

## C-Factor (Cover Management Factor)

The C-factor represents the ratio of soil loss from land cropped and managed under specific conditions to the corresponding loss from clean-tilled, continuous fallow. It accounts for the effects of:

* **Vegetative Cover:** Type of vegetation, canopy density, and height.
* **Surface Residue:** Amount of crop residue or mulch on the soil surface.
* **Tillage Practices:** Type and timing of tillage operations.
* **Land Management:** Crop rotations, cover crops, and other management strategies.

A C-factor of 1.0 indicates conditions equivalent to clean-tilled continuous fallow (maximum erosion potential from cover perspective), while values closer to 0 indicate very effective cover and management practices that minimize erosion.

### Simplified C-Factor Estimation

For this interactive tool, the C-factor is estimated based on:

1.  **Broad Land Use Category:** A base C-factor (`base_c`) is selected from a predefined table.
2.  **Canopy Cover (%):** User-defined percentage of soil surface covered by plant canopies.
3.  **Ground Residue Cover (%):** User-defined percentage of soil surface covered by crop residues or mulch.

The `base_c` values for different land uses are illustrative and represent typical conditions:

| Land Use Category                               | Base C-Factor (`base_c`) |
| :---------------------------------------------- | :----------------------- |
| Forest (Well-stocked, undisturbed)              | 0.003                    |
| Pasture/Rangeland (Good grass cover)            | 0.01                     |
| Cropland - Conservation Tillage (High residue >60%) | 0.10                     |
| Cropland - Conventional Tillage (Low residue <20%)  | 0.40                     |
| Urban/Construction (Bare, compacted soil)       | 0.8                      |
| Fallow (Clean-tilled, no residue)               | 1.0                      |
| Orchard/Vineyard (with cover crop)              | 0.15                     |
| Orchard/Vineyard (clean-tilled)                 | 0.5                      |

**Note on Canopy and Residue Adjustments:**

The Python code then *conceptually* adjusts this `base_c` based on the user-input `Canopy Cover (%)` and `Ground Residue Cover (%)` sliders. The principle is:
* Higher canopy and residue cover generally lead to a lower effective C-factor.
* The exact mathematical adjustment in the provided script is a simplification. For instance:
    * If the `base_c` is 1.0 (like Fallow), the canopy and residue directly reduce this value based on their coverage and an assumed effectiveness.
    * For other land uses, the script attempts to model deviations from a 'typical' cover assumed within the `base_c` value. Increasing canopy/residue beyond this 'typical' level reduces the C-factor, while decreasing it increases the C-factor, ensuring it stays within the 0.001 to 1.0 range.

This approach is for demonstration; precise C-factor calculation often involves more detailed sub-factors related to prior land use, canopy, surface cover, surface roughness, and soil moisture, often within frameworks like RUSLE2.

In [ ]:
land_use_c_base_values = {
    "Forest (Well-stocked, undisturbed)": 0.003,
    "Pasture/Rangeland (Good grass cover)": 0.01,
    "Cropland - Conservation Tillage (High residue >60%)": 0.10, # e.g., No-till
    "Cropland - Conventional Tillage (Low residue <20%)": 0.40,  # e.g., Moldboard plow
    "Urban/Construction (Bare, compacted soil)": 0.8,
    "Fallow (Clean-tilled, no residue)": 1.0,
    "Orchard/Vineyard (with cover crop)": 0.15,
    "Orchard/Vineyard (clean-tilled)": 0.5
}

def calculate_c_factor(land_use, canopy_cover_percent, ground_residue_percent):
    """
    Estimates C-factor based on land use, canopy, and residue.
    This is a highly simplified model.
    """
    base_c = land_use_c_base_values.get(land_use, 1.0) # Default to 1.0 if unknown

    # Apply reduction factors for canopy and residue
    # Max reduction effect at 100% cover.

    canopy_effect = 1.0 - (canopy_cover_percent / 100.0 * 0.8) # Canopy can reduce C by up to 80% of (1-base_c) or base_c
    residue_effect = 1.0 - (ground_residue_percent / 100.0 * 0.9) # Residue can reduce C by up to 90%
    # For bare soil (C=1), the reduction is directly proportional.

    # Simplified interaction:
    if base_c == 1.0: # For initially bare soil
        effective_c = base_c * canopy_effect * residue_effect
    else:

        reduction_from_canopy = (canopy_cover_percent / 100.0) * base_c * 0.5 # Max 50% reduction from base due to canopy
        reduction_from_residue = (ground_residue_percent / 100.0) * base_c * 0.7 # Max 70% reduction from base due to residue
        # Ensure C doesn't go below a minimum (e.g. forest floor still has a C)

        # C_final = base_c * (1 - 0.008 * canopy_cover_percent) * (1 - 0.009 * ground_residue_percent)

        c_val = base_c
        # If canopy is better than 'typical' for that class (assume typical is ~50% if not forest/pasture)
        if canopy_cover_percent > 50:
            c_val *= (1 - (canopy_cover_percent - 50)/100 * 0.5) # reduce by up to 25%
        elif canopy_cover_percent < 50  and not ("Forest" in land_use or "Pasture" in land_use):
            c_val *= (1 + (50 - canopy_cover_percent)/100 * 0.5) # increase by up to 25%

        if ground_residue_percent > 50:
            c_val *= (1 - (ground_residue_percent - 50)/100 * 0.7) # reduce by up to 35%
        elif ground_residue_percent < 50 and not ("Forest" in land_use or "Pasture" in land_use) :
            c_val *= (1 + (50 - ground_residue_percent)/100 * 0.7) # increase by up to 35%

        effective_c = np.clip(c_val, 0.001, 1.0) # Ensure C is within [0.001, 1.0]

    return effective_c


## P-Factor (Support Practice Factor)

The P-factor, also known as the erosion control practice factor or support practice factor, reflects the effects of practices that reduce erosion by modifying the flow pattern, grade, or direction of surface runoff. It is the ratio of soil loss with a specific support practice to the soil loss with up-and-down slope farming.

A P-factor of 1.0 is assigned to land with no support practices or where farming is done up-and-down the slope. Values less than 1.0 indicate that the practice reduces soil loss.

### Simplified P-Factor Estimation in this Tool

For this tool, the P-factor is estimated based on:

1.  **Type of Support Practice:** Selected from a predefined list.
2.  **Land Slope (%):** User-defined average slope of the land, which is particularly important for contouring.

The P-factor values are based on common estimates:

| Support Practice                             | Slope (%)         | P-Factor | Notes                                                                 |
| :------------------------------------------- | :---------------- | :------- | :-------------------------------------------------------------------- |
| None (Up and Down Slope Farming)             | Any               | 1.0      | Baseline condition.                                                   |
| Contouring                                   | 0 - 2%            | 0.6      | Effectiveness varies with slope.                                      |
|                                              | >2 - 7%           | 0.5      |                                                                       |
|                                              | >7 - 12%          | 0.6      |                                                                       |
|                                              | >12 - 18%         | 0.7      |                                                                       |
|                                              | >18 - 24%         | 0.8      |                                                                       |
|                                              | >24%              | 0.9      | Becomes less effective and potentially detrimental on very steep slopes. |
| Strip Cropping (Alternate Crops, Contoured)  | Any (assumed on contour) | 0.35     | Represents an average; actual value depends on strip width & rotation. |
| Terraces (Broad-base, Graded)                | Any (design specific) | 0.15     | Represents an average; depends on terrace type, spacing, and condition. |
| Grassed Waterways                            | N/A for P-factor  | 0.9 (conceptual) | Primarily controls ephemeral gully erosion, often used with other practices. Its P-factor representation here is highly conceptual. |

**Note on P-Factor Values:**

* The values for contouring are slope-dependent, reflecting how its effectiveness changes.
* Values for strip cropping and terraces are averages. Actual P-factors would depend on detailed design specifications.
* Grassed waterways are critical for managing concentrated flow but their direct representation as a single P-factor in the same way as contouring or terracing is a simplification for this tool; they often act in conjunction with other practices.

In [ ]:
support_practices_p_values = {
    "None (Up and Down Slope Farming)": 1.0,
    "Contouring": "slope_dependent", # Special case
    "Strip Cropping (Alternate Crops, Contoured)": 0.35, # Average value
    "Terraces (Broad-base, Graded)": 0.15, # Average value
    "Grassed Waterways": 0.9 # Applied with other practices, mainly affects ephemeral gully
}

def calculate_p_factor(practice, slope_percent):
    """
    Estimates P-factor based on support practice and slope.
    This is a simplified model.
    """
    if practice == "Contouring":
        # P-factor for contouring is slope-dependent (values from Wischmeier & Smith, 1978 for example)
        if slope_percent <= 2: return 0.6
        elif slope_percent <= 7: return 0.5
        elif slope_percent <= 12: return 0.6
        elif slope_percent <= 18: return 0.7
        elif slope_percent <= 24: return 0.8
        else: return 0.9 # Contouring less effective on very steep slopes
    else:
        return support_practices_p_values.get(practice, 1.0)

## Interactive Plotting Function

In [ ]:
# C-Factor Widgets
land_use_options = list(land_use_c_base_values.keys())
c_land_use_dropdown = widgets.Dropdown(options=land_use_options, value=land_use_options[2], description="Land Use (C):")
c_canopy_slider = widgets.FloatSlider(value=70, min=0, max=100, step=5, description="Canopy (%):", readout_format='.0f')
c_residue_slider = widgets.FloatSlider(value=80, min=0, max=100, step=5, description="Residue (%):", readout_format='.0f')

# P-Factor Widgets
support_practice_options = list(support_practices_p_values.keys())
p_practice_dropdown = widgets.Dropdown(options=support_practice_options, value=support_practice_options[0], description="Practice (P):")
p_slope_slider = widgets.FloatSlider(value=5, min=0, max=50, step=0.5, description="Slope (%):", readout_format='.1f')


# --- Create Interactive UI ---
# Organize widgets using VBox and HBox for better layout

c_factor_title = HTML("<h3>C-Factor (Cover and Management)</h3>")
c_factor_widgets = VBox([c_land_use_dropdown, c_canopy_slider, c_residue_slider])

p_factor_title = HTML("<h3>P-Factor (Support Practices)</h3>")
p_factor_widgets = VBox([p_practice_dropdown, p_slope_slider])

# Interactive output
interactive_plot = interactive(plot_interactive_cp,
                               land_use=c_land_use_dropdown,
                               canopy_cover=c_canopy_slider,
                               ground_residue=c_residue_slider,
                               support_practice=p_practice_dropdown,
                               slope=p_slope_slider)

# Display the UI
ui = VBox([
    HBox([VBox([c_factor_title, c_factor_widgets]), VBox([p_factor_title, p_factor_widgets])]),
    interactive_plot.children[-1] # The output plot
])

display(ui)